# Clickbait Detection using Gradient Boosting & UMAP Embeddings

## Εισαγωγή
Στο παρόν notebook παρουσιάζεται η διαδικασία εκπαίδευσης και αξιολόγησης ενός **Gradient Boosting Classifier** για την αναγνώριση clickbait τίτλων.

Για την αναπαράσταση των κειμένων, χρησιμοποιούμε διανύσματα (embeddings) που έχουν μειωθεί σε **500 διαστάσεις** μέσω της τεχνικής **UMAP**. Τα δεδομένα έχουν ήδη διαχωριστεί σε **Train**, **Validation** και **Test** sets, εξασφαλίζοντας ότι δεν υπάρχει διαρροή πληροφορίας (data leakage).

## Στόχοι
1. **Φόρτωση Προ-επεξεργασμένων Δεδομένων**: Χρήση των αρχείων Parquet (`train_umap_500`, `valid_umap_500`, `test_umap_500`).
2. **Βελτιστοποίηση Υπερπαραμέτρων (Hyperparameter Tuning)**: Χρήση της βιβλιοθήκης **Optuna** για την εύρεση των βέλτιστων παραμέτρων του μοντέλου.
3.  **Εκπαίδευση Champion Model (Φάση 2)**: Το βέλτιστο μοντέλο επανεκπαιδεύεται στο συνδυαστικό σύνολο (Train + Validation) για τη μέγιστη δυνατή αξιοποίηση της πληροφορίας.
4.  **Τελική Αξιολόγηση**: Το μοντέλο δοκιμάζεται στο **Test Set**, το οποίο παρέμεινε "αθέατο" κατά τη διάρκεια της βελτιστοποίησης. Καταγράφονται μετρικές όπως F1-Score, Accuracy, Precision-Recall Curve, και Χρόνος Εκτέλεσης.

In [ ]:
import pandas as pd
import numpy as np
import optuna
import mlflow
import sys
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score

# Ρύθμιση Paths για εντοπισμό των βοηθητικών αρχείων και δεδομένων
# Θεωρούμε ότι το notebook βρίσκεται στο: Models/Random Forest - Gradient Boosting/
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import mlflow_helper

project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
DATA_FOLDER = os.path.join(project_root, 'data', 'clean', 'umap')

print(f"Project Root: {project_root}")
print(f"Data Folder: {DATA_FOLDER}")

## Φόρτωση Δεδομένων (Data Loading)

Φορτώνουμε τα αρχεία `.parquet` που περιέχουν τα embeddings (500 διαστάσεις) και τα labels.
* **Train Set**: Χρησιμοποιείται για την εκπαίδευση των υποψήφιων μοντέλων στο Optuna.
* **Validation Set**: Χρησιμοποιείται από το Optuna για την αξιολόγηση κάθε trial και την επιλογή των βέλτιστων παραμέτρων.
* **Test Set**: Κρατείται "κλειστό" και χρησιμοποιείται **μόνο** στο τέλος για την αξιολόγηση του τελικού μοντέλου.

In [ ]:
def load_split_data(data_path):
    files = {
        "Train": "train_umap_500.parquet",
        "Valid": "valid_umap_500.parquet",
        "Test":  "test_umap_500.parquet"
    }

    loaded_data = {}
    # Λίστα πιθανών ονομάτων για τη στήλη στόχο
    possible_label_cols = ['labels', 'label', 'target', 'class', 'is_clickbait']

    print(f"Έναρξη διαδικασίας φόρτωσης από: {data_path}")

    for name, filename in files.items():
        file_path = os.path.join(data_path, filename)

        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Το αρχείο {filename} δεν βρέθηκε.")

        # Προσπάθεια ανάγνωσης με πολλαπλά engines για συμβατότητα
        try:
            df = pd.read_parquet(file_path, engine='fastparquet')
        except:
            df = pd.read_parquet(file_path, engine='pyarrow')

        # 1. Εντοπισμός Features (στήλες που ξεκινούν με 'umap_')
        feature_cols = [c for c in df.columns if c.startswith("umap_")]

        # Fallback: Αν δεν βρεθούν, παίρνουμε όλες εκτός από τα πιθανά labels
        if not feature_cols:
            feature_cols = [c for c in df.columns if c not in possible_label_cols]

        # 2. Εντοπισμός Labels
        label_col = None
        for col in possible_label_cols:
            if col in df.columns:
                label_col = col
                break

        # Αν δεν υπάρχει label εντός του parquet, ψάχνουμε για εξωτερικό αρχείο
        if label_col is None:
            remaining = [c for c in df.columns if c not in feature_cols]
            if len(remaining) == 1:
                label_col = remaining[0]
                print(f"   [{name}] Label column autodetected: '{label_col}'")
            else:
                prefix = filename.split('_')[0]
                ext_path = os.path.join(data_path, f"{prefix}_labels.csv")
                if os.path.exists(ext_path):
                    print(f"   [{name}] Labels loaded from external file: {prefix}_labels.csv")
                    df_labels = pd.read_csv(ext_path)
                    y = df_labels.iloc[:, 0].values.astype(int)
                    X = df[feature_cols].values.astype(np.float32)
                    loaded_data[name] = (X, y)
                    print(f"   [{name}] Loaded Successfully: X={X.shape}, y={y.shape}")
                    continue
                else:
                    raise ValueError(f"Σφάλμα στο {name}: Δεν βρέθηκε label (ούτε εσωτερικά, ούτε εξωτερικά).")

        if label_col:
            if label_col in feature_cols:
                feature_cols.remove(label_col)
            y = df[label_col].values.astype(int)

        X = df[feature_cols].values.astype(np.float32)

        if len(X) != len(y):
             raise ValueError(f"Ασυμφωνία διαστάσεων στο {name}: X={len(X)}, y={len(y)}")

        loaded_data[name] = (X, y)
        print(f"   [{name}] Loaded Successfully: X={X.shape}, y={y.shape}")

    return loaded_data["Train"], loaded_data["Valid"], loaded_data["Test"]

# Εκτέλεση Φόρτωσης
(X_train, y_train), (X_val, y_val), (X_test, y_test) = load_split_data(DATA_FOLDER)

## 4. Βελτιστοποίηση Υπερπαραμέτρων (Optuna Objective)

Ορίζουμε τη συνάρτηση `objective`, η οποία καλείται από το Optuna σε κάθε δοκιμή (trial).
Η συνάρτηση εκτελεί τα εξής:
1.  Επιλέγει ένα σετ υπερπαραμέτρων από τον ορισμένο χώρο αναζήτησης (Search Space).
2.  Εκπαιδεύει το μοντέλο στο Training Set.
3.  **Καταγράφει τον χρόνο εκπαίδευσης**, καθώς αποτελεί κρίσιμη μετρική για την παραγωγική λειτουργία.
4.  Αξιολογεί το μοντέλο στο Validation Set.
5.  Καταγράφει όλες τις πληροφορίες στο **MLflow** με μοναδικό όνομα trial.

In [ ]:
def objective(trial, X_tr, y_tr, X_v, y_v):
    # Ορισμός χώρου αναζήτησης
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 300),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_samples_split": trial.suggest_int("min_samples_split", 10, 50),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "random_state": 42
    }

    model = GradientBoostingClassifier(**params)

    # Μέτρηση Χρόνου Εκπαίδευσης
    start_time = time.time()
    model.fit(X_tr, y_tr)
    training_time = time.time() - start_time

    # Αξιολόγηση
    preds = model.predict(X_v)
    f1 = f1_score(y_v, preds)
    acc = accuracy_score(y_v, preds)

    # Καταγραφή μετρικών για το συγκεκριμένο trial
    metrics = {
        "val_f1": f1,
        "val_accuracy": acc,
        "training_time_sec": training_time
    }

    # Αποστολή στο MLflow (Nested Run)
    mlflow_helper.log_optuna_trial(
        trial,
        params,
        metrics,
        model,
        run_name_prefix="GB_Trial"
    )

    return f1

## 5. Εκτέλεση Πειράματος & Τελική Αξιολόγηση

Η διαδικασία χωρίζεται σε δύο Run στο MLflow για καθαρότερη εποπτεία:
* **Run 1 (Tuning):** Περιέχει όλες τις δοκιμές του Optuna.
* **Run 2 (Champion):** Περιέχει μόνο το τελικό, βέλτιστο μοντέλο.

Το τελικό μοντέλο εκπαιδεύεται εκ νέου στο σύνολο `Train + Validation` και αξιολογείται στο `Test Set`.

In [ ]:
EXPERIMENT_NAME = "Clickbait_GradientBoosting_UMAP_Final"
mlflow_helper.setup_mlflow(EXPERIMENT_NAME)

print(f"\nΈναρξη Πειράματος: {EXPERIMENT_NAME}")

# --- ΦΑΣΗ 1: Hyperparameter Tuning ---
print("\n[ΦΑΣΗ 1] Αναζήτηση Βέλτιστων Παραμέτρων (Optuna)...")

with mlflow.start_run(run_name="GB_Hyperparameter_Tuning") as tuning_run:
    mlflow.log_param("dataset", "UMAP_500")
    mlflow.log_param("model_type", "GradientBoosting")

    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="maximize", sampler=sampler)

    # Εκτέλεση 15 δοκιμών
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=15)

    print(f"Best Params found: {study.best_params}")
    print(f"Best Val F1: {study.best_value:.4f}")

# --- ΦΑΣΗ 2: Champion Model Training ---
print("\n[ΦΑΣΗ 2] Εκπαίδευση & Αποθήκευση Champion Model...")

with mlflow.start_run(run_name="GB_Champion_Model") as final_run:

    # Ανάκτηση παραμέτρων
    best_params = study.best_params
    best_params["random_state"] = 42

    mlflow.log_params(best_params)
    mlflow.log_param("model_type", "GB_Champion")
    mlflow.log_param("dataset", "UMAP_500")

    final_model = GradientBoostingClassifier(**best_params)

    # Ένωση Train και Validation sets
    X_full_train = np.concatenate((X_train, X_val))
    y_full_train = np.concatenate((y_train, y_val))

    # Εκπαίδευση και χρονομέτρηση
    start_t = time.time()
    final_model.fit(X_full_train, y_full_train)
    final_train_time = time.time() - start_t
    print(f"Total Training Time: {final_train_time:.2f} sec")

    # Αποθήκευση μοντέλου
    mlflow.sklearn.log_model(final_model, artifact_path="champion_model")

    # Τελική Αξιολόγηση στο Test Set
    print("Υπολογισμός τελικών μετρικών στο Test Set...")
    mlflow_helper.evaluate_and_log_metrics(
        final_model,
        X_test,
        y_test,
        prefix="test",
        training_time=final_train_time
    )

    print(f"\nΟλοκληρώθηκε επιτυχώς. Run ID: {final_run.info.run_id}")
    print(f"Αποτελέσματα διαθέσιμα στο MLflow Run: 'GB_Champion_Model'")

## Συμπεράσματα

Η διαδικασία ολοκληρώθηκε. Το τελικό μοντέλο εκπαιδεύτηκε με τις βέλτιστες παραμέτρους και αξιολογήθηκε σε άγνωστα δεδομένα (Test Set).

**Επόμενα Βήματα:**
* Έλεγχος των γραφημάτων (Confusion Matrix, ROC Curve) στο MLflow UI.
* Σύγκριση των αποτελεσμάτων με άλλα μοντέλα για την επιλογή της τελικής λύσης.